In [2]:
from datetime import datetime, timedelta
import config
import sqlite3

def dict_factory(cursor, row):
    """Convert database row to dictionary"""
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

def get_db_connection():
    """Get database connection with row factory"""
    conn = sqlite3.connect(config.DATABASE_PATH)
    conn.row_factory = dict_factory
    return conn


def create_epochs(epoch_data):
    """Create multiple epochs at once, ignoring conflicts"""
    conn = get_db_connection()
    cursor = conn.cursor()
    
    try:
        cursor.executemany(
            '''
            INSERT INTO epochs (start_time, end_time, lock_start, lock_end, status) 
            VALUES (?, ?, ?, ?, ?)
            ON CONFLICT(start_time) DO NOTHING
            ''',
            epoch_data
        )
        conn.commit()
    finally:
        conn.close()

def generate_epochs():
    epochs = []
    num_epochs = int(24*60*60/config.EPOCH_DURATION_SECONDS) #calculating number of epochs to be enough for 36 hours
    current_start_time = datetime.now().replace(minute=0, second=0, microsecond=0)

    for _ in range(num_epochs):
        epoch_end_time = current_start_time + timedelta(seconds=config.EPOCH_DURATION_SECONDS)

        lock_start_time = current_start_time - timedelta(seconds=config.EPOCH_LOCK_SECONDS)
        lock_end_time = current_start_time
            
        epochs.append((
            current_start_time.strftime('%Y-%m-%d %H:%M:%S'),
            epoch_end_time.strftime('%Y-%m-%d %H:%M:%S'),
            lock_start_time.strftime('%Y-%m-%d %H:%M:%S'),
            lock_end_time.strftime('%Y-%m-%d %H:%M:%S'),
            'new'
        ))
        current_start_time = epoch_end_time
    
    create_epochs(epochs)


generate_epochs()


In [11]:
from datetime import datetime, timedelta
import config
import sqlite3



def dict_factory(cursor, row):
    """Convert database row to dictionary"""
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

def get_db_connection():
    """Get database connection with row factory"""
    conn = sqlite3.connect(config.DATABASE_PATH)
    conn.row_factory = dict_factory
    return conn


def create_epochs(epoch_data):
    """Create multiple epochs at once, ignoring conflicts and returning the ids"""
    conn = get_db_connection()
    cursor = conn.cursor()
    
    try:
        cursor.executemany(
            '''
            INSERT INTO epochs (start_time, end_time, lock_start, lock_end, status) 
            VALUES (?, ?, ?, ?, ?)
            ON CONFLICT(start_time) DO NOTHING
            ''',
            epoch_data
        )
        conn.commit()
        
        # Retrieve the IDs of the inserted epochs (those that were inserted, not already existing)
        cursor.execute('SELECT id, start_time FROM epochs WHERE start_time IN (?, ?, ?)', 
                       [epoch[0] for epoch in epoch_data])  # Assuming epoch_data is a list of tuples
        
        epoch_ids = {row[1]: row[0] for row in cursor.fetchall()}
        return epoch_ids
    finally:
        conn.close()

def create_rounds(round_data):
    """Create multiple rounds at once, associating them with epochs"""
    conn = get_db_connection()
    cursor = conn.cursor()
    
    try:
        cursor.executemany(
            '''
            INSERT INTO rounds (epoch_id, round_number, start_time, end_time, lockdown_end_time, status)
            VALUES (?, ?, ?, ?, ?, ?)
            ''',
            round_data
        )
        conn.commit()
    finally:
        conn.close()

def generate_epochs():
    epochs = []
    rounds = []
    num_epochs = int(24 * 60 * 60 / config.EPOCH_DURATION_SECONDS)  # Number of epochs to cover 36 hours
    current_start_time = datetime.now().replace(minute=0, second=0, microsecond=0)

    for _ in range(num_epochs):
        epoch_end_time = current_start_time + timedelta(seconds=config.EPOCH_DURATION_SECONDS)
        lock_start_time = current_start_time - timedelta(seconds=config.EPOCH_LOCK_SECONDS)
        lock_end_time = current_start_time
        
        # Store the epoch data
        epochs.append((
            current_start_time.strftime('%Y-%m-%d %H:%M:%S'),
            epoch_end_time.strftime('%Y-%m-%d %H:%M:%S'),
            lock_start_time.strftime('%Y-%m-%d %H:%M:%S'),
            lock_end_time.strftime('%Y-%m-%d %H:%M:%S'),
            'new'
        ))
        
        # Calculate rounds based on dynamic round length and lockdown period
        total_rounds_duration = config.EPOCH_DURATION_SECONDS  # Total time for all rounds in an epoch
        round_duration = total_rounds_duration / config.NUM_ROUNDS  # Length of each round
        
        for round_number in range(1, config.NUM_ROUNDS + 1):
            round_start_time = current_start_time + timedelta(seconds=(round_number - 1) * round_duration)
            round_end_time = current_start_time + timedelta(seconds=round_number * round_duration)
            
            # Calculate lockdown end time after round
            lockdown_end_time = round_end_time + timedelta(seconds=config.ROUND_LOCK_SECONDS)
            
            # Store the round data with a reference to the epoch (epoch_id will be added after epochs are inserted)
            rounds.append((
                None,  # Placeholder for epoch_id
                round_number,
                round_start_time.strftime('%Y-%m-%d %H:%M:%S'),
                round_end_time.strftime('%Y-%m-%d %H:%M:%S'),
                lockdown_end_time.strftime('%Y-%m-%d %H:%M:%S'),
                'new'
            ))
        
        # Update current start time for the next epoch
        current_start_time = epoch_end_time
    
    # First, insert the epochs
    epoch_ids = create_epochs(epochs)
    
    # Now, update round data with epoch_ids and create the rounds
    for i, round_data in enumerate(rounds):
        epoch_start_time = round_data[2]  # Use round's start time to find the matching epoch
        if epoch_start_time in epoch_ids:
            round_data = (epoch_ids[epoch_start_time],) + round_data[1:]  # Assign the correct epoch_id
            rounds[i] = round_data
        else:
            print(f"Epoch for round {i} not found, skipping round.")  # Optionally log or handle the case
    
    # Insert the rounds
    create_rounds(rounds)

generate_epochs()


AttributeError: module 'config' has no attribute 'NUM_ROUNDS'

In [14]:
from datetime import datetime, timedelta
import config
import sqlite3

def dict_factory(cursor, row):
    """Convert database row to dictionary"""
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

def get_db_connection():
    """Get database connection with row factory"""
    conn = sqlite3.connect(config.DATABASE_PATH)
    conn.row_factory = dict_factory
    return conn

def insert_epoch(start_time, end_time, lock_start, lock_end, status):
    """Insert epoch, ignoring conflicts"""
    conn = get_db_connection()
    cursor = conn.cursor()
    try:
        cursor.execute(
            """INSERT INTO epochs (start_time, end_time, lock_start, lock_end, status)
                VALUES (?, ?, ?, ?, ?)
                ON CONFLICT(start_time) DO NOTHING
            """,
            (start_time, end_time, lock_start, lock_end, status)
        )
        conn.commit()
        return cursor.lastrowid
    finally:
        conn.close()

def insert_rounds(rounds_data):
    """Insert rounds"""
    conn = get_db_connection()
    cursor = conn.cursor()
    
    try:
        cursor.executemany(
            'INSERT INTO rounds (epoch_id, start_time, end_time, lock_start, lock_end, starting_price, ending_price, status) VALUES (?, ?, ?, ?, ?, ?, ?, ?)',
            rounds_data
        )
        conn.commit()
    finally:
        conn.close()


def generate_epochs_and_rounds():
    epochs = []
    num_epochs = int(24*60*60/config.EPOCH_DURATION_SECONDS) #calculating number of epochs to be enough for 36 hours
    current_start_time = datetime.now().replace(minute=0, second=0, microsecond=0)

    for _ in range(num_epochs):
        epoch_end_time = current_start_time + timedelta(seconds=config.EPOCH_DURATION_SECONDS)

        lock_start_time = current_start_time - timedelta(seconds=config.EPOCH_LOCK_SECONDS)
        lock_end_time = current_start_time
            
        epochs.append((
            current_start_time.strftime('%Y-%m-%d %H:%M:%S'),
            epoch_end_time.strftime('%Y-%m-%d %H:%M:%S'),
            lock_start_time.strftime('%Y-%m-%d %H:%M:%S'),
            lock_end_time.strftime('%Y-%m-%d %H:%M:%S'),
            'new'
        ))
        current_start_time = epoch_end_time
    
    for epoch in epochs:
        epoch_id = insert_epoch(epoch[0],epoch[1],epoch[2],epoch[3],epoch[4])
        if epoch_id != 0:
            round_starts = datetime.strptime(epoch[0], "%Y-%m-%d %H:%M:%S") # epoch_start
            epoch_end = datetime.strptime(epoch[1], "%Y-%m-%d %H:%M:%S")
            round_duration = timedelta(seconds=config.EPOCH_DURATION_SECONDS//config.ROUNDS_COUNT)
            rounds = [] 
            for i in range(config.ROUNDS_COUNT):
                lockdown_start_seconds = int(round_duration.total_seconds() * (1-config.ROUND_LOCK_PERCENTAGE)) 
                lockdown_start = round_starts + timedelta(seconds=lockdown_start_seconds)
                
                lockdown_end = round_starts + round_duration  # Lockdown ends when the round ends
                
                round_end = lockdown_start
                
                # If it's the last round, ensure it ends at epoch_ends
                if i == config.ROUNDS_COUNT - 1:
                    lockdown_end = epoch_end

                rounds.append((
                    epoch_id,
                    round_starts.strftime('%Y-%m-%d %H:%M:%S'),
                    round_end.strftime('%Y-%m-%d %H:%M:%S'),
                    lockdown_start.strftime('%Y-%m-%d %H:%M:%S'),
                    lockdown_end.strftime('%Y-%m-%d %H:%M:%S'),
                    0,
                    0,
                    'new'
                ))

                round_starts = lockdown_end 
            insert_rounds(rounds)
            

generate_epochs_and_rounds()


In [ ]:
epoch_starts = datetime.now().replace(minute=0, second=0, microsecond=0)
epoch_ends = epoch_starts + timedelta(seconds=config.EPOCH_DURATION_SECONDS)
round_count = 9
round_duration = timedelta(seconds=config.EPOCH_DURATION_SECONDS//round_count)
LOCKDOWN_PERCENTAGE = 0.6
#lockdown_duration = round_duration * LOCKDOWN_PERCENTAGE

print(f"epoch starts {epoch_starts.strftime('%Y-%m-%d %H:%M:%S')} epoch ends {epoch_ends.strftime('%Y-%m-%d %H:%M:%S')}")
round_starts = epoch_starts
for i in range(round_count):

    lockdown_start_seconds = int(round_duration.total_seconds() * (1-LOCKDOWN_PERCENTAGE)) 
    lockdown_start = round_starts + timedelta(seconds=lockdown_start_seconds)
    
    lockdown_end = round_starts + round_duration  # Lockdown ends when the round ends
    
    round_end = lockdown_start
    
    # If it's the last round, ensure it ends at epoch_ends
    if i == round_count - 1:
        lockdown_end = epoch_ends

    print(f"round {i+1} start {round_starts}, round ends {round_end}, "
          f"lockdown starts {lockdown_start}, lockdown ends {lockdown_end}")
    
    round_starts = lockdown_end 



epoch starts 2025-03-07 16:00:00 epoch ends 2025-03-07 16:10:00
round 1 start 2025-03-07 16:00:00, round ends 2025-03-07 16:00:26, lockdown starts 2025-03-07 16:00:26, lockdown ends 2025-03-07 16:01:06
round 2 start 2025-03-07 16:01:06, round ends 2025-03-07 16:01:32, lockdown starts 2025-03-07 16:01:32, lockdown ends 2025-03-07 16:02:12
round 3 start 2025-03-07 16:02:12, round ends 2025-03-07 16:02:38, lockdown starts 2025-03-07 16:02:38, lockdown ends 2025-03-07 16:03:18
round 4 start 2025-03-07 16:03:18, round ends 2025-03-07 16:03:44, lockdown starts 2025-03-07 16:03:44, lockdown ends 2025-03-07 16:04:24
round 5 start 2025-03-07 16:04:24, round ends 2025-03-07 16:04:50, lockdown starts 2025-03-07 16:04:50, lockdown ends 2025-03-07 16:05:30
round 6 start 2025-03-07 16:05:30, round ends 2025-03-07 16:05:56, lockdown starts 2025-03-07 16:05:56, lockdown ends 2025-03-07 16:06:36
round 7 start 2025-03-07 16:06:36, round ends 2025-03-07 16:07:02, lockdown starts 2025-03-07 16:07:02, lock

In [ ]:
for i in range(round_count):
    # Calculate the time when the lockdown starts (after 50% of the round time)
    lockdown_start = round_starts + round_duration * LOCKDOWN_PERCENTAGE
    # Lockdown lasts for the same percentage (50% of the round time)
    lockdown_duration = round_duration * LOCKDOWN_PERCENTAGE
    lockdown_end = lockdown_start + lockdown_duration
    
    # Round end time
    round_end = round_starts + round_duration
    
    # If it's the last round, ensure it ends at epoch_ends
    if i == round_count - 1:
        round_end = epoch_ends

    # Print the round and lockdown times
    print(f"round {i+1} start {round_starts}, round ends {round_end}, "
          f"lockdown starts {lockdown_start}, lockdown ends {lockdown_end}")
    
    # Update round_starts for the next round, after the lockdown ends
    round_starts = lockdown_end  # The next round starts after the lockdown ends

In [ ]:
epoch_starts = datetime.now().replace(minute=0, second=0, microsecond=0)
epoch_ends = epoch_starts + timedelta(seconds=config.EPOCH_DURATION_SECONDS)
round_count = 10
round_duration = timedelta(seconds=config.EPOCH_DURATION_SECONDS//round_count)
LOCKDOWN_PERCENTAGE = 50

print(f"epoch starts {epoch_starts.strftime('%Y-%m-%d %H:%M:%S')} epoch ends {epoch_ends.strftime('%Y-%m-%d %H:%M:%S')}")
round_starts = epoch_starts
for i in range(round_count):
    if i == round_count - 1:
        end_time = epoch_ends
    else: 
        end_time = round_starts + round_duration

    print(f"round {i+1} start {round_starts}, round ends {end_time}")
    round_starts = end_time  # Update start for next round



In [ ]:
def process_epoch_lock_start():
    print("a")
